<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Bert_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


     |████████████████████████████████| 778kB 3.3MB/s 
     |████████████████████████████████| 3.0MB 16.6MB/s 
     |████████████████████████████████| 890kB 24.5MB/s 
     |████████████████████████████████| 1.1MB 44.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=92b2cc6eaf9ba085b90d8bf60e1fb0334b7ada4d40014dbf6c69742757f760f6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
#import the libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [3]:
#use pandas to load the data set
df=pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',delimiter='\t',header=None)

In [4]:
batch=df[:2000]

In [5]:
from transformers import BertModel, BertTokenizer

In [6]:
model_class=BertModel

In [7]:
tokenizer_class=BertTokenizer

In [11]:
pretrained_weights='bert-base-uncased'

In [12]:
#load pretrained mode
tokenizer=tokenizer_class.from_pretrained(pretrained_weights)

In [13]:
model=model_class.from_pretrained(pretrained_weights)

In [14]:
tokenized=batch[0].apply(lambda x:tokenizer.encode(x,add_special_tokens=True))

In [15]:
max_len=0
#find the maximum length of the list
for i in tokenized.values:
  if len(i) > max_len :
    max_len=len(i) 

#concatenate the lists with zeroes for remaining length
padded=np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) 

In [16]:
padded.shape

(2000, 59)

In [17]:
attention_mask=np.where(padded!=0,1,0)
attention_mask.shape

(2000, 59)

In [18]:
#Construct a tensor with padded attribute
input_ids=torch.tensor(padded)
input_ids

tensor([[  101,  1037, 18385,  ...,     0,     0,     0],
        [  101,  4593,  2128,  ...,     0,     0,     0],
        [  101,  2027,  3653,  ...,     0,     0,     0],
        ...,
        [  101,  2023,  2028,  ...,     0,     0,     0],
        [  101,  1999,  1996,  ...,     0,     0,     0],
        [  101,  1996,  3185,  ...,     0,     0,     0]])

In [19]:
#convert the attention array to torch tenosr
attention_mask=torch.tensor(attention_mask)

In [21]:
#Embed the sentence or the input ids
with torch.no_grad():
  last_hidden_states=model(input_ids,attention_mask=attention_mask)
  #The outout is encoded senttence with 768 dim, 
  #the output is rep in 3d way by #number of sentence,#tokens in sentence,sentene embedding dimension
  #here output is (2000,66,768) 

In [22]:
#Access the first token from each of the sentences
features=last_hidden_states[0][:,0,:].numpy()

In [24]:
labels=batch[1]

In [25]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [26]:
#perform logistic regression
lr_clf = LogisticRegression(C=5.2)

lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
#evalute the score
lr_clf.score(test_features, test_labels)

0.822